In [1]:
# Put in names here
result_image_path = "results/finetune_edges2shoes/test_latest/images"

In [2]:
import torch
import os
import re
from PIL import Image
import cv2
import numpy as np
from skimage.measure import compare_ssim as ssim

In [3]:
def make_image_path_pairs(dir):
    image_pairs = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir

    fake_regex = r'(.*)_fake_B.png'
    real_regex = r'(.*)_real_B.png'
    fake_paths = {}
    real_paths = {}

    for root, _, fnames in sorted(os.walk(dir)):
        for fname in fnames:
            path = os.path.join(root, fname)
            match = re.search(fake_regex, fname)
            if match:
                key = match[1]
                if key in real_paths:
                    real_path = real_paths[key]
                    image_pairs.append((path, real_path))
                    del real_paths[key]
                    continue
                fake_paths[key] = path
                continue

            match = re.search(real_regex, fname)
            if match:
                key = match[1]
                if key in fake_paths:
                    fake_path = fake_paths[key]
                    image_pairs.append((fake_path, path))
                    del fake_paths[key]
                    continue
                real_paths[key] = path
    return image_pairs

In [4]:
# image_pairs_PIL = [(Image.open(fake).convert('RGB'), Image.open(real).convert('RGB')) for fake, real in image_path_pairs]
image_path_pairs = make_image_path_pairs(result_image_path)
image_pairs = [(cv2.imread(fake), cv2.imread(real)) for fake, real in image_path_pairs]

In [5]:
def MAE(imageA, imageB): # same as torch.nn.L1Loss, which is put out
    abs_error = np.sum(abs(imageA.astype("float") - imageB.astype("float")))
    mean_abs_error = abs_error / float(imageA.shape[0] * imageB.shape[1] * imageB.shape[2])
    return mean_abs_error

def RMSE(imageA, imageB): # Root-mean-squared error, equivalent of L2
    sq_error = np.sum((imageA.astype("float") - imageB.astype("float"))**2)
    mse = sq_error / float(imageA.shape[0] * imageB.shape[1] * imageB.shape[2])
    return np.sqrt(mse)

In [6]:
l1_distances = []
l2_distances = []
ssims = []
for fake, real in image_pairs:
    l1_distances.append(MAE(fake, real))
    l2_distances.append(RMSE(fake, real))
    ssims.append(ssim(fake, real, multichannel=True))

avg_l1_dist = sum(l1_distances)/len(l1_distances)
avg_l2_dist = sum(l2_distances)/len(l2_distances)
avg_ssim = sum(ssims)/len(ssims)
print("Average mean average error (l1 loss) between the {} image pairs: {}".format(len(l1_distances), avg_l1_dist))
print("Average root mean squared error (l2 loss) between the {} image pairs: {}".format(len(l2_distances), avg_l2_dist))
print("Average SSIM between the {} image pairs: {}".format(len(ssims), avg_ssim))

Average mean average error (l1 loss) between the 294 image pairs: 35.24483533240771
Average root mean squared error (l2 loss) between the 294 image pairs: 56.928349348058234
Average SSIM between the 294 image pairs: 0.486556817458017
